# PRELAB: Quantum-Key Distribution with BB84
Name

Date

## Pre-Lab

#### Worked Problem #1 - entangled photon generation
For example: Quantum states of outputs of Type 0, 1, 2 SPDC.
For example: If I tell you the pump has $\lambda_p$ and the signal is $\lambda_s$, can you tell me $\lambda_i$ of the idler? (Energy conservation)

#### Worked Problem #2 - One time pad encryption
In order to use OTP, you need a key that's the same length as the message. Additionally, you need a new key for every message you want to encrypt and send: the protocol is only secure when each pad is used only once.

Encrypting and decrypting a message is simple: bitwise XOR with the key. If you XOR the raw message with the key, you get the ciphertext. Then if you XOR the cypher test with the key, you get the initial message back.

In [1]:
import numpy as np

LENGTH = 10;
# Generate a random message
mess = np.random.randint(low = 0, high = 2, size = (LENGTH,))

# Generate a random key
key = np.random.randint(low = 0, high = 2, size = (LENGTH,))

# Generate the cypher text by performing bitwise XOR on the key and the message
cypher = mess^key

# Recover the original message by performing bitwise XOR on the cypher
# and the key
recover = cypher^key

print("Message: % s" %(mess))
print("Key    : % s" %(key))
print("Now XOR those to get the cyphertext:")
print("Cypher : % s" %(cypher))
print("To recover the original message, you XOR the key and the cypher")
print("Key    : % s" %(key))
print("Cypher : % s" %(cypher))
print("The recovered message after decryption is")
print("Decrypt: % s" %(recover))
print("Compare with original message")
print("Message: % s" %(mess))

Message: [0 0 0 1 1 1 1 1 0 0]
Key    : [1 0 0 0 1 0 1 0 1 0]
Now XOR those to get the cyphertext:
Cypher : [1 0 0 1 0 1 0 1 1 0]
To recover the original message, you XOR the key and the cypher
Key    : [1 0 0 0 1 0 1 0 1 0]
Cypher : [1 0 0 1 0 1 0 1 1 0]
The recovered message after decryption is
Decrypt: [0 0 0 1 1 1 1 1 0 0]
Compare with original message
Message: [0 0 0 1 1 1 1 1 0 0]


#### Worked Problem #3 - Alice and Bob estimate the error
My code isn't necessarily the most pedagogically useful - do we want to make this easier for students to parse?

In [2]:
import numpy as np
import random

# How many photons does Alice send?
NN = 1000

# What is the transmission between Alice and Bob? 
# Transmission of 1 implies there's no loss at all
# Transmission of 0 implies Eve is blocking the Alice to Bob channel
TT = 0.9

# How often does Eve listen in? 0 is never, 1 is always
FE = 0.1

# Initialize arrays

# What bits did Alice's transmitted message send?
bits_tx = np.zeros((NN)) - 1
# What polarization did Alice use?
pol_tx = np.zeros((NN)) - 1

# Did the photon make it?
photon_rx = np.zeros((NN)) - 1
# Bob's measurement basis
pol_rx = np.zeros((NN)) - 1
# Bob's received photons
bits_rx = np.zeros((NN)) - 1

# Did Eve interefere?
Eve_rx = np.zeros((NN)) - 1
Eve_pol = np.zeros((NN)) - 1
Eve_bit = np.zeros((NN)) - 1



for i in range (NN):
    # Alice generates a random bit to send
    bit_A = random.randint(0,1)
    bits_tx[i] = bit_A
    # Now she selects a basis
    bas_A = random.randint(0,1)
    pol_tx[i] = bas_A
    
   # Now decide if Eve does something
    if random.random() <= FE: # if Eve -DOES- decide to eavesdrop & send
        Eve_rx[i] = 1 # Record that this is an occurence of Eve eavesdropping
        Eve_bas = random.randint(0,1) # She randomly selects a basis
        Eve_pol[i] = Eve_bas
        if Eve_bas == bas_A: # If her basis matches Alice's
            Eve_bit[i] = bit_A # Then she recovers the bit perfectly
        else:
            Eve_bit[i] = random.randint(0,1) # otherwise, it's a coin toss   
        bit_rx = Eve_bit[i] # Now the bit Bob receives will be the bit Eve sends
        bas_rx = Eve_bas # In the basis that Eve sends
    else: # If Eve isn't measuring and sending out photons
        pho_E = 0 # Mark this as no Eve involvement
        bit_rx = bit_A # The bit Bob receives is ths bit Alice sends
        bas_rx = bas_A # in the basis in which Alice sent it
        
    # If the photon makes it through the loss channel
    if random.random() <= TT:
        photon_rx[i] = 1
        bas_B = random.randint(0,1) # Bob randomly selects a basis
        pol_rx[i] = bas_B
        if bas_B == bas_rx:
            bit_B = bit_rx
        else:
            bit_B = random.randint(0,1)
        bits_rx[i] = bit_B
    else:
        photon_rx[i] = 0;

# now do some processing


# First limit ourselves to cases where Alice and Bob used the same bases
ix = np.array([np.where(pol_tx == pol_rx)])
bits_tx_sifted = np.array(bits_tx[ix])
bits_tx_sifted = bits_tx_sifted[0,0]
bits_rx_sifted = np.array(bits_rx[ix])
bits_rx_sifted = bits_rx_sifted[0,0]

# Now look bit error rate (BER)
errors = (bits_tx_sifted != bits_rx_sifted).astype(int)
errors_total = errors.sum()
usable_key = ix.size - errors_total
BER = 1.0/ix.size * errors.sum()

# What did Eve get?
ixe = np.array([np.where((Eve_rx ==1) & (Eve_pol == pol_tx))])


print("Alice sent Bob % d bits"% NN)
print("Bob received % d bits"% photon_rx.sum())
print("They used the same basis in %d of those bits" %ix.size)
print("This resulted in a total of %d bits without errors" %usable_key)
print("The BER was %10.2E" %BER)
print("Eve got %d bits" %ixe.size)



Alice sent Bob  1000 bits
Bob received  909 bits
They used the same basis in 407 of those bits
This resulted in a total of 396 bits without errors
The BER was   2.70E-02
Eve got 48 bits


Based on this code, we see that as Eve listens in more often, she receives more bits of the key. Alice and Bob can see a difference in the BER as Eve listens in more often. When Eve listens every time, the BER is > 20%. When Eve never listens in, the BER is 0 (ie no errors).
_Do we want them to plot BER as a function of how often Eve listens in?_

#### Worked Problem #4 - ThorLabs QKD analogy
QKD's security is based on the No Cloning Theorem, and the classical light source sends more than one photon per pulse. So, Eve can siphon off some light and then you're hosed.